In [ ]:
import pandas as pd
import numpy as np  
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from datetime import datetime
import mappings as mp

In [ ]:
parquet_name = 'data/yellow_taxi_jan_2021.parquet'
csv_name = 'data/ny_taxi_location_lookup.csv'
data_url="https://d37ci6vzurychx.cloutaxi_df_cleanront.net/trip-data/yellow_tripdata_2021-01.parquet"
lookup_url="https://d37ci6vzurychx.cloutaxi_df_cleanront.net/misc/taxi+_zone_lookup.csv"

In [ ]:
# ONE TIME DOWNLOAD  
# os.system(f'wget -O {parquet_name} {data_url}')
# os.system(f'wget -O {csv_name} {lookup_url}')

In [ ]:
# READ IN DATA 
taxi_df_orig = pd.read_parquet(parquet_name)


In [ ]:
# CHECK MEMORY USAGE 
memory_usage = taxi_df_orig.memory_usage(deep=True).sum() / (1024 ** 2)  # Convert to megabytes
print(f"Memory usage of DataFrame: {memory_usage:.2f} MB")

# MAPPINGS

In [ ]:
# IMPORT MAPPING DICTIONARY
mapping = mp.mapping_dict
print(f'data type:{type(mapping)}', f'an example value: {mapping["payment_type"][3]}')
#dir(mappings)

# BASIC INFO AND STATS

In [ ]:
taxi_df = taxi_df_orig.copy()
taxi_df.columns = taxi_df.columns.str.lower()
columns = taxi_df.columns
columns 

In [ ]:
taxi_df.info()

#### REPLACE STORE AND FORWARD FLAG WITH A NUMERIC VALUE FOR ANALYSIS PURPOSES

In [ ]:
print(taxi_df['store_and_fwd_flag'].unique())  
# Replace "Y" with 1 and "N" with 2 
taxi_df['store_and_fwd_flag'] = taxi_df['store_and_fwd_flag'].replace({"Y": 1, "N": 2}).astype(float) 
print(taxi_df['store_and_fwd_flag'].unique())

#### LOOK AT NA, NULL, AND ZERO COUNTS

In [ ]:
def nan_null_zero_datatypes(df):
    have_values = [(df[col].notna() & (df[col] != 0)).sum() for col in df.columns]
    nans = [df[col].isna().sum() for col in df.columns]
    blanks = [(df[col] == ' ').sum() for col in df.columns]
    zeros = [(df[col] == 0).sum() for col in df.columns]
    data_types = [df[col].apply(lambda x: type(x)).unique() for col in df.columns]
    len_data_types = [len(df[col].apply(lambda x: type(x)).unique()) for col in df.columns] 
    

    df_clean_check = pd.DataFrame({
        'Column Name': df.columns,
        'have_values': have_values,
        '# NAs': nans,
        '# Blanks': blanks,
        '# Zeros': zeros,
        '# Data Types': len_data_types,
        'Data Types': data_types
    })

    return df_clean_check

In [ ]:
taxi_df_info = nan_null_zero_datatypes(taxi_df)
taxi_df_info

## BASIC STATS

In [ ]:
taxi_df.describe().round(2)

#### UNDERSTANDING THE DISCRETE VALUE COLUMNS

In [ ]:
# PRINT UNIQUE VALUES FOR COLUMNS WITH DISCRETE VALUES
print("Value counts for columns with discrete values:")
for col in taxi_df.columns:
    unique_values = taxi_df[col].unique()
    if len(unique_values) < 20:
        print(f"{col}:")
        for value in unique_values:
            count = (taxi_df[col] == value).sum()
            print(f"    {value}: {count}")
            
        # OR FOLLOWING THE FIRST PRINT
        #value_counts = taxi_df[col].value_counts()
        #for value, count in value_counts.items():
        #    print(f"    {value}: {count}")

## ADD NEW COLUMNS 
Adding now because duration will be used to select some records for removal. 

- duration_minute
- holiday flag 
- math check --> Total Amount - Fare and all Fees/ Charges

EVENTUALLY WANT TO ADD 
- Weather - temp and percipitation (rain/snow) flag
- Daytime - flag Y if between sunrise and sunset for that day

In [ ]:
# BREAK OUT DROP OFF DATE INTO SEPARATE COLUMNS
taxi_df['drop_off_day'] = taxi_df['tpep_dropoff_datetime'].dt.day
taxi_df['drop_off_hour'] = taxi_df['tpep_dropoff_datetime'].dt.hour
taxi_df['drop_off_dow'] = taxi_df['tpep_dropoff_datetime'].dt.dayofweek

# ADD PICK UP DATE INTO SEPARATE COLUMNS
taxi_df['pick_up_day'] = taxi_df['tpep_pickup_datetime'].dt.day
taxi_df['pick_up_hour'] = taxi_df['tpep_pickup_datetime'].dt.hour
taxi_df['pick_up_dow'] = taxi_df['tpep_pickup_datetime'].dt.dayofweek

# ADD DURATION IN MINUTES COLUMN
taxi_df['duration_minute'] = (
    (taxi_df['tpep_dropoff_datetime'] - taxi_df['tpep_pickup_datetime'])
    .apply(lambda x: round(x.total_seconds() / 60))
    .astype(int)
)

taxi_df.head()

In [ ]:
# ADD HOLIDAY FLAG COLUMN

# List of US holidays in Jan 2021 
holidays = ['2021-01-01', '2021-01-18'] 

# Add a holiday flag column
taxi_df['is_holiday'] = (
    taxi_df['tpep_dropoff_datetime']
    .apply(lambda x: x.strftime('%Y-%m-%d') in holidays)
)

# Look at records with holidays
holiday = taxi_df[taxi_df['is_holiday'] == True]
print(f"Records with holidays: {holiday.shape[0]}")
holiday.tail()

In [ ]:
# CHECK IF THE AMOUNTS ADD UP

money_cols = ['payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'total_amount', 'math_check']

taxi_df['math_check'] = (
    taxi_df['total_amount'] -
    taxi_df['extra'] -
    taxi_df['mta_tax'] -
    taxi_df['tip_amount'] -
    taxi_df['tolls_amount'] -
    taxi_df['improvement_surcharge'] -
    taxi_df['congestion_surcharge'] -
    taxi_df['fare_amount']
).round(2)


math_check_df = taxi_df[taxi_df['math_check'] != 0]
print(math_check_df.shape[0])

# QUESTIONABLE DATA 
### Look into these further

- **airport_fee** -- only 5 records with 0 and rest nan --> drop this columns
- **negatives** 
- **NaNs** -- - 98352 records with NA in passenger_count, rate_code_id, store_and_fwd_flag, and congestion_surcharge and 0 in Payment type<br> --> account for **.07%** of the records
- **trip distance** -- 19952 records with trip_distance = 0 
- **vendor_id** -- 10291 records with vendor_id = 6
- **passengers** -- 26726 records with 0 passengers
- **ratecodeid** -- 36 records with ratecodeid = 99 
- **total amount** 452 records with 0 total amount
- negative amounts - check to see if all negative taxes correspond to negative amounts -- guessing they are refunds

### DROP AIRPORT FEE COLUMN

In [ ]:
# DROP THE AIRPORT FEE COLUMN
taxi_df = taxi_df.drop('airport_fee', axis=1)
taxi_df.shape 

### NEGATIVE AMOUNTS
First suspected that these were refunds but the records have non negative trip distances. 
The rows with neg fare amount will be dropped. That captures all but few rows with other neg values. 

In [ ]:
# LOOK AT COUNTS OF NEGATIVE VALUES
print('COUNTS OF NEGATIVE VALUES PER COLUMN')
for col in taxi_df.columns:
    if taxi_df[col].dtype != 'datetime64[us]':
        negatives = (taxi_df[col] < 0).sum()
        if negatives > 0: 
            print(f"    {col}: {negatives}")

In [ ]:
# CONFIRM THAT ALMOST ALL OF THE NEGATIVE TAXES, CHARGES ETC OCCUR WITH THE NEGATIVE FARES
neg_fare = taxi_df[taxi_df['fare_amount'] < 0]
for col in neg_fare.columns:
    if neg_fare[col].dtype != 'datetime64[us]':
        negatives = (neg_fare[col] < 0).sum()
        if negatives > 0: 
            print(f"    {col}: {negatives}")

In [ ]:
# STATS FOR NEGATIVE FARES DATAFRAME
neg_fare.describe().round(2)

In [ ]:
nan_neg_fare = neg_fare.isna().sum()
print(nan_neg_fare)

### CHECK NaN RECORDS 
All NaNs are in the same rows. These rows will be dropped. 

In [ ]:
# CHECK IF ALL NaNs ARE IN THE SAME ROWS
na_check = taxi_df[taxi_df['passenger_count'].isna()]
print(f"Rows with NAs: {na_check.shape[0]} ")
na_check.describe().round(2)    

In [ ]:
na_check[na_check['trip_distance'] == 0].shape

### RECORDS WITH TRIP DISTANCE = 0 
These records will be kept but will be excluded from any distance stats. 

In [ ]:
# RECORDS WITH 0 TRIP DISTANCE
zero_distance = taxi_df[taxi_df['trip_distance'] == 0]
print(f"Records with 0 trip distance: {zero_distance.shape[0]} ")
zero_distance.describe().round(2)

### RECORDS WITH INVALID VENDOR ID (6) 
These all occur with the NaN combo. They will be dropped with the NaN drop. 

In [ ]:
# RECORDS WITH VENDOR ID = 6
vendor_6 = taxi_df[taxi_df['vendorid'] == 6]
print(f"Records with vendor id = 6: {vendor_6.shape[0]} ")
vendor_6.describe().round(2)

### RECORDS WITH NO PASSENGERS
This could be a data entry error. These records will be kept but excluded from per passenger stats

In [ ]:
# RECORDS WITH 0 PASSANGERS
zero_passangers = taxi_df[taxi_df['passenger_count'] == 0]
print(f"Records with 0 passengers: {zero_passangers.shape[0]} ")
zero_passangers.describe().round(2)

In [ ]:
# ONlY 5 RECORDS WITH 0 PASSENGERS AND ZERO FARE
zero_passangers[zero_passangers['total_amount'] == 0].shape

### RECORDS WITH INVALID RATECODEID 
There are only 36 records. These will be deleted since there are not that many and it will make analysis easier later. 
Could try and figure out how the rate code is determined and see if the cost per time or distance matches the other rate codes... but that is too much work for 36 records. 

In [ ]:
# RECORDS WITH RATECODEID = 99
ratecodeid_99 = taxi_df[taxi_df['ratecodeid'] == 99]
print(f"Records with ratecodeid = 99: {ratecodeid_99.shape[0]} ")
ratecodeid_99.describe().round(2)

In [ ]:
# ONlY 6 RECORDS WITH INVALID RATECODEID AND ZERO FARE
ratecodeid_99[ratecodeid_99['total_amount'] == 0].shape

In [ ]:
nan_ratecodeid_99 = ratecodeid_99.isna().sum()
print(nan_ratecodeid_99)

### CHECK DATES AND EXCLUDE RECORDS WITH DROP OF DATE OUTSIDE OF JANUARY
Since the rate code is selected at drop off, I am assuming that the fare is also associated with the drop off. Records will be kept as long as the drop off time is in the month of Jan. 

In [ ]:
# CHECK IF RECORDS OUTSIDE OF JAN 2021 - BASED ON DROP OFF ONLY 
min_date = datetime(2021, 1, 1)
max_date = datetime(2021, 2, 1)
# Filter for dates less than the certain date
filtered_do_taxi_df = taxi_df[(taxi_df['tpep_dropoff_datetime'] < min_date) | (taxi_df['tpep_dropoff_datetime'] >= max_date)]
print(f"Records outside of Jan 2021: {filtered_do_taxi_df.shape[0]} ")
filtered_do_taxi_df.tail()

In [ ]:
nan_filtered_do_taxi_df = filtered_do_taxi_df[filtered_do_taxi_df.isna().any(axis=1)]
nan_filtered_do_taxi_df.head()

In [ ]:
nan_filtered_do_taxi_df = filtered_do_taxi_df.isna().sum()
print(nan_filtered_do_taxi_df)

In [ ]:
# CHECK IF THESE INCLUDE NEGATIVE VALUES
for col in filtered_do_taxi_df.columns:
    if filtered_do_taxi_df[col].dtype != 'datetime64[us]':
        negatives = (filtered_do_taxi_df[col] < 0).sum()
        if negatives > 0: 
            print(f"    {col}: {negatives}")
        else:
            print(f"    {col}: no negatives")
    

In [ ]:
# CHECK IF RECORDS OUTSIDE OF JAN 2021 - BASED ON PICK UP AND DROP OFF 
min_date = datetime(2021, 1, 1)
max_date = datetime(2021, 2, 1)
# Filter for dates less than the certain date
filtered_dopu_taxi_df_clean = taxi_df[((taxi_df['tpep_pickup_datetime'] < min_date) & (taxi_df['tpep_dropoff_datetime'] < min_date)) 
                      | ((taxi_df['tpep_pickup_datetime'] > max_date) & (taxi_df['tpep_dropoff_datetime'] > max_date))]
print(f"Records outside of Jan 2021: {filtered_dopu_taxi_df_clean.shape[0]} ")
filtered_dopu_taxi_df_clean.head(20)

# DROP ROWS BASED ON ABOVE
1. With negative fare amount - 6769 records 114 of which fall into another bucket (NaN) 
2. With invalid ratecodeid - 36 
3. With NaNs - 98352 records 
4. With drop off time not in Jan 2021 - 137 records 4 of which fall into another bucket (NaN)

In [ ]:
# EXPECTED ROWS TO DROP
print(f'Expected dropped rows: {6769 - 114 + 36 + 98352 + 137 - 4}') 

In [ ]:
taxi_df_clean = (
    taxi_df[taxi_df['fare_amount'] >= 0]
    .query('ratecodeid != 99')
    .query('@min_date <= tpep_dropoff_datetime < @max_date')
    .dropna()
)
print(f'Dropped Rows: {1369769 - taxi_df_clean.shape[0]}')


# OUTLIERS

In [ ]:
taxi_df_clean.describe().round(2)

In [ ]:
# VERY LONG TRIP DISTANCE 
print(taxi_df_clean.shape[0])
taxi_df_clean.drop(taxi_df_clean[taxi_df_clean['trip_distance'] >= 500].index, inplace = True)
print(taxi_df_clean.shape[0])

In [ ]:
# UNBELIEVABLE FARE AMOUNTS 
# The records with trip distance 0 or 2 miles and high fare amounts are likely to be errors or money laundering. 
print(taxi_df_clean[(taxi_df_clean['fare_amount'] >= 300) & (taxi_df_clean['trip_distance'] <= 10)].shape[0])
taxi_df_clean[(taxi_df_clean['fare_amount'] >= 300) & (taxi_df_clean['trip_distance'] <= 10)].head()

In [ ]:
# VERY HIGH FARE AMOUNTS AS IT SKEWS THE DATA - THESE OUTLIERS CAN BE LOOKED AT INDIVIDUALLY
print(taxi_df_clean[taxi_df_clean['total_amount'] >= 500].shape[0])

In [ ]:
# LOOK AT RECORDS WITH FARE AMOUNT GREATER THAN $300, 0 DISTANCE, AND DURATION LESS THAN 2 MINUTES 
# These are likely errors or money laundering  -  1297 records
zero_distance2 = (
    taxi_df_clean[(taxi_df_clean['trip_distance'] == 0) 
    & (taxi_df_clean['duration_minute'] < 2)
    & (taxi_df_clean['total_amount'] > 50)]
)
print(zero_distance2.shape[0])  
zero_distance2.sort_values(by=['total_amount'], ascending=False).head(20)

In [ ]:
# DROP THE RECORDS WITH FARE AMOUNT GREATER THAN $300, 0 DISTANCE, AND DURATION LESS THAN 2 MINUTES
taxi_df_clean = taxi_df_clean[~((taxi_df_clean['trip_distance'] == 0) & (taxi_df_clean['duration_minute'] < 2) & (taxi_df_clean['total_amount'] > 50))]
taxi_df_clean.shape[0]

In [ ]:
# DROP THE RECORDS WITH TOTAL FARE AMOUNT GREATER THAN $500
taxi_df_clean = taxi_df_clean[~(taxi_df_clean['total_amount'] > 500)]
taxi_df_clean.shape[0]

In [ ]:
# LONG DURATION TRIPS  - THE DRIVER LIKELY FORGOT TO TURN OFF THE METER
# LEAVE THEM IN THE DATASET BUT EXCLUDE FROM DURATION ANALYSIS
print(taxi_df_clean[(taxi_df_clean['duration_minute'] >= 500) & (taxi_df_clean['trip_distance'] < 200)].shape[0])
print(taxi_df_clean[(taxi_df_clean['duration_minute'] >= 1000)].shape[0])
taxi_df_clean[taxi_df_clean['duration_minute'] >= 1000].sort_values(by=['trip_distance'], ascending=False).head(20)

# NO CHARGE (payment_type = 3) and DISPUTE TRIPS (payment_type = 4)
Review some records

In [ ]:
no_charge = taxi_df_clean[taxi_df_clean['payment_type'] == 3]   
print(f'No charge records: {no_charge.shape[0]}')
no_charge.describe().round(2)

In [ ]:
dispute = taxi_df_clean[taxi_df_clean['payment_type'] == 4]   
print(f'Dispute records: {dispute.shape[0]}')
dispute.describe().round(2)

# EXPORT CLEANED DATA TO FILE

In [ ]:
taxi_df_clean.to_parquet('data/yellow_taxi_jan_2021_clean.parquet')
taxi_df_clean.to_pickle("data/yellow_taxi_jan_2021_clean.pkl")

In [ ]:
taxi_df_clean.info()

In [ ]:
categorical_cols = ['passenger_count', 'vendorid', 'ratecodeid', 'store_and_fwd_flag','payment_type', 'pulocationid', 'dolocationid','drop_off_day', 'drop_off_hour','drop_off_dow', 'pick_up_day', 'pick_up_hour', 'pick_up_dow','is_holiday']
print(len(categorical_cols))

numeric_cols = ['trip_distance', 'duration_minute', 'total_amount', 'fare_amount', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'mta_tax', 'extra', 'congestion_surcharge', 'math_check']
print(len(numeric_cols))

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(12, 8))

for i, column in enumerate(numeric_cols):
    taxi_var = taxi_df_clean[column]
    mean = taxi_var.mean()
    std_dev = taxi_var.std()
    
    row = i // 3
    col = i % 3
    
    sns.histplot(taxi_var, bins=3000, ax=axs[row, col])
    axs[row, col].set_title(column.capitalize())
    
    # Set x-axis limits based on min and max values in the column
    axs[row, col].set_xlim(mean - 3*std_dev, mean + 3*std_dev)
    
    axs[row, col].axvline(taxi_var.mean(), color='red', label='Mean')
    axs[row, col].axvline(taxi_var.mean() - taxi_var.std(), color='green', linestyle='--', label='Std Dev')
    axs[row, col].axvline(taxi_var.mean() + taxi_var.std(), color='green', linestyle='--')

# Add a single legend for all four plots
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=(1, 1), loc='right', ncol=1)

# Add a title for the whole figure
fig.suptitle('Taxi Metrics Histograms', fontsize=16, y=1.05)

# Add some padding between the subplots
plt.subplots_adjust(hspace=1)

# display the plot
plt.show()

In [ ]:
sns.boxplot(x='drop_off_day', y='fare_amount', data=taxi_df_clean, hue='drop_off_dow', palette='rainbow', showfliers=False, legend=False)
plt.xlabel('Drop-off Day of Month')
plt.ylabel('Total Amount')
plt.title('Boxplot of Total Amount by Drop-off Day of Week')
plt.show()

In [ ]:
sns.violinplot(x='drop_off_dow', y='fare_amount', data=taxi_df, hue='drop_off_dow', palette='rainbow', legend=False)
plt.xlabel('Drop-off Day of Week')
plt.ylabel('Total Amount')
plt.title('Boxplot of Total Amount by Drop-off Day of Week')
plt.show()